In [1]:
import pandas as pd
import requests
from datetime import datetime

In [4]:
import accessKeys

# Acces to SQL Database

In [5]:
schema = "cities_database"
host = "localhost"
user = "root"
password = accessKeys.mySQL_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [6]:
cities_from_sql = pd.read_sql("city", con=connection_string)
city_df = cities_from_sql[["city_id", "city_name", "latitude", "longitude"]]

In [7]:
latitudes = city_df['latitude'].to_list()
longitudes = city_df['longitude'].to_list()
city_ids = city_df['city_id'].to_list()

# API

## Get Airports

In [8]:
def Get_Airports_per_Location(lats,lons,city_ids):
    list_of_airport_dfs = []
    for lat,lon,city_id in zip(lats,lons,city_ids):

        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        querystring = {"lat":lat,"lon":lon,"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}
        headers = {
            "x-rapidapi-key": accessKeys.API_airport_key,
            "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers, params=querystring)

        airport_df = pd.json_normalize(response.json()['items'])
        airport_df['city_id'] = city_id
        list_of_airport_dfs.append(airport_df)

    return pd.concat(list_of_airport_dfs,axis=0)

In [ ]:
airport_df = Get_Airports_per_Location(lats=latitudes, lons=longitudes, city_ids=city_ids)

In [94]:
airport_df

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon,city_id
0,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699,1
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889,1
0,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229,2
0,LFPB,LBG,Paris -Le Bourget,-Le Bourget,Paris,FR,Europe/Paris,48.96940,2.441390,3
1,LFPO,ORY,Paris -Orly,-Orly,Paris,FR,Europe/Paris,48.72530,2.359440,3
2,LFPG,CDG,Paris Charles de Gaulle,Charles de Gaulle,Paris,FR,Europe/Paris,49.01280,2.549999,3
0,ESSB,BMA,Stockholm -Bromma,-Bromma,Stockholm,SE,Europe/Stockholm,59.35440,17.941698,4
1,ESSA,ARN,Stockholm -Arlanda,-Arlanda,Stockholm,SE,Europe/Stockholm,59.65190,17.918600,4
0,RJTT,HND,Tokyo,Tokyo,Tokyo,JP,Asia/Tokyo,35.55230,139.780000,5
0,LIRA,CIA,Roma Ciampino–G. B. Pastine,Ciampino–G. B. Pastine,Roma,IT,Europe/Rome,41.79940,12.594900,6


## Get Flights

In [10]:
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2025-06-20T00:00/2025-06-20T12:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"x-rapidapi-key": accessKeys.API_flight_key,
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'message': 'You have exceeded the MONTHLY quota for Tier 2 on your current plan, BASIC. Upgrade your plan at https://rapidapi.com/aedbx-aedbx/api/aerodatabox'}


In [ ]:
flights_df = pd.json_normalize(response.json()["arrivals"])

In [97]:
flights_df

,number,callSign,status,codeshareStatus,isCargo,departure.airport.icao,departure.airport.iata,departure.airport.name,departure.airport.timeZone,departure.scheduledTime.utc,...,arrival.quality,aircraft.reg,aircraft.modeS,aircraft.model,airline.name,airline.iata,airline.icao,departure.runwayTime.utc,departure.runwayTime.local,departure.runway
0,HV 5666,TRA9Y,Arrived,IsOperator,False,GCLP,LPA,Gran Canaria Island,Atlantic/Canary,2025-06-19 18:05Z,...,"[Basic, Live]",PH-YHA,4866ED,Airbus A321-200 (Sharklets),Transavia,HV,TRA,NaN,NaN,NaN
1,HV 6706,TRA83Q,Arrived,IsOperator,False,GCFV,FUE,Fuerteventura Island,Atlantic/Canary,2025-06-19 18:15Z,...,"[Basic, Live]",PH-YHY,4866DE,Airbus A321-200 (Sharklets),Transavia,HV,TRA,NaN,NaN,NaN
2,HV 5586,TRA5586,Arrived,IsOperator,False,LFMN,NCE,Nice,Europe/Paris,2025-06-19 20:35Z,...,"[Basic, Live]",PH-HSK,485122,Boeing 737-800 (winglets),Transavia,HV,TRA,NaN,NaN,NaN
3,HV 6886,TRA85K,Arrived,IsOperator,False,BIKF,KEF,Reykjavik,Atlantic/Reykjavik,2025-06-19 19:00Z,...,"[Basic, Live]",PH-HSI,485009,Boeing 737-800 (winglets),Transavia,HV,TRA,2025-06-19 19:30Z,2025-06-19 19:30+00:00,NaN
4,HV 6598,TRA6598,Arrived,IsOperator,False,LGKR,CFU,Kerkyra Island,Europe/Athens,2025-06-19 19:25Z,...,"[Basic, Live]",PH-HSG,484F6D,Boeing 737-800 (winglets),Transavia,HV,TRA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,KL 984,KLM50W,Arrived,IsOperator,False,EGLC,LCY,London,Europe/London,2025-06-20 09:00Z,...,"[Basic, Live]",PH-EXV,485871,Embraer 190,KLM,KL,KLM,NaN,NaN,09
266,KL 1088,KLM70Y,Arrived,IsOperator,False,EGFF,CWL,Cardiff,Europe/London,2025-06-20 08:50Z,...,"[Basic, Live]",PH-EXN,4855D0,Embraer 175,KLM,KL,KLM,NaN,NaN,NaN
267,HV 5794,TRA5794,Arrived,IsOperator,False,LGPZ,PVK,Preveza/Lefkada,Europe/Athens,2025-06-20 06:40Z,...,"[Basic, Live]",PH-YHB,4866EE,Airbus A321-200 (Sharklets),Transavia,HV,TRA,NaN,NaN,NaN
268,U2 7962,EJU46JE,Arrived,IsOperator,False,EPKK,KRK,Kraków,Europe/Warsaw,2025-06-20 07:45Z,...,"[Basic, Live]",OE-LSV,440075,Airbus A321-200 (Sharklets),easyJet,U2,EZY,2025-06-20 08:22Z,2025-06-20 10:22+02:00,25


In [98]:
list_of_icaos = airport_df["icao"].to_list()
list_city_ids = airport_df["city_id"].to_list()

In [ ]:
def Get_Flight_Data(icao_codes, city_ids):
    flight_data_dfs = []
    for icao,city_id in zip(icao_codes, city_ids):

        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/2025-06-20T00:00/2025-06-20T12:00"

        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

        headers = {
            "x-rapidapi-key": accessKeys.API_flight_key,
            "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        if(response.status_code != 200):
            continue

        flights_df = pd.json_normalize(response.json()["arrivals"])
        flights_df['city_id'] = city_id
        flights_df["arrival_icao"] = icao
        flight_data_dfs.append(flights_df)

    return pd.concat(flight_data_dfs,axis=0)

In [100]:
flight_data_df = Get_Flight_Data(list_of_icaos, list_city_ids)

In [101]:
flight_data_df

,number,callSign,status,codeshareStatus,isCargo,departure.airport.icao,departure.airport.iata,departure.airport.name,departure.airport.timeZone,departure.quality,...,arrival.runwayTime.utc,arrival.runwayTime.local,arrival.runway,departure.terminal,arrival.terminal,arrival.gate,arrival.baggageBelt,departure.checkInDesk,departure.gate,city_id
0,DJ 6228,SRR6228,Approaching,IsOperator,False,EDDK,CGN,Cologne,Europe/Berlin,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,5O 5DN,FPO5DN,Arrived,IsOperator,False,LFPG,CDG,Paris,Europe/Paris,"[Basic, Live]",...,2025-06-20 03:50Z,2025-06-20 05:50+02:00,25R,NaN,NaN,NaN,NaN,NaN,NaN,1
2,HU 489,CHH489,Arrived,IsOperator,False,ZBAA,PEK,Beijing,Asia/Shanghai,[Basic],...,NaN,NaN,NaN,2,1,N02,A3,NaN,NaN,1
3,XQ 966,SXS3Z,Arrived,IsOperator,False,LTBJ,ADB,İzmir,Europe/Istanbul,"[Basic, Live]",...,NaN,NaN,NaN,I,1,N02,A4,10,101,1
4,EW 6900,EWG6900,Arrived,IsOperator,False,NaN,NaN,Unknown,NaN,[],...,2025-06-20 04:30Z,2025-06-20 06:30+02:00,25L,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,FA 315,SFR315,Unknown,IsOperator,False,FALA,HLA,Jo'anna,Africa/Johannesburg,[Basic],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
27,EK 772,UAE772,Unknown,IsOperator,False,OMDB,DXB,Dubai,Asia/Dubai,[Basic],...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,9
28,5Z 834,NaN,Unknown,Unknown,False,FAOR,JNB,Jo'anna,Africa/Johannesburg,[Basic],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
29,TK 44,NaN,Expected,IsOperator,False,LTFM,IST,Istanbul,Europe/Istanbul,"[Basic, Live]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E-F,A9,9


# Send Data back to SQL

## Clean Airport Dataframe

In [ ]:
# Check if all the datatypes are correct
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 0
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   icao              15 non-null     object 
 1   iata              15 non-null     object 
 2   name              15 non-null     object 
 3   shortName         15 non-null     object 
 4   municipalityName  15 non-null     object 
 5   countryCode       15 non-null     object 
 6   timeZone          15 non-null     object 
 7   location.lat      15 non-null     float64
 8   location.lon      15 non-null     float64
 9   city_id           15 non-null     int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 1.3+ KB


In [18]:
airport_df_fin = airport_df[["icao", "iata", "name", "countryCode", "timeZone", "location.lat", "location.lon", "city_id"]]

In [ ]:
airport_df_fin

In [20]:
airport_df_fin.to_sql("airport", 
                  if_exists="append", 
                  con=connection_string, 
                  index=False)

15

## Clean Flights Dataframe

In [69]:
flight_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1226 entries, 0 to 42
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   number                         1226 non-null   object
 1   callSign                       1096 non-null   object
 2   status                         1226 non-null   object
 3   codeshareStatus                1226 non-null   object
 4   isCargo                        1226 non-null   bool  
 5   departure.airport.icao         1205 non-null   object
 6   departure.airport.iata         1205 non-null   object
 7   departure.airport.name         1226 non-null   object
 8   departure.airport.timeZone     1205 non-null   object
 9   departure.quality              1226 non-null   object
 10  arrival.scheduledTime.utc      1226 non-null   object
 11  arrival.scheduledTime.local    1226 non-null   object
 12  arrival.revisedTime.utc        975 non-null    object
 13  arrival.re

In [ ]:
# Number, Callsign, status, departure.airport.icao, airline.name, arrival.scheduledTime.local

In [ ]:
flights_df_fin = flight_data_df[["number", "callSign", "status", "departure.airport.icao", "airline.name", "arrival.scheduledTime.local", "arrival_icao"]]

In [ ]:
flights_df_fin.to_sql("flights", 
                  if_exists="append", 
                  con=connection_string,
                  index=False)